In [3]:
!pip install -r "requirments.txt"

In [16]:
from facenet_pytorch import MTCNN
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import BinaryCrossentropy
import tensorflow as tf]
import os

## Loading the model 

In [13]:
model = load_model(os.path.join("Completed model" ,"SmileOrNotModel.keras"), compile=False)
model.compile(loss=BinaryCrossentropy(), optimizer='adam', metrics=['accuracy'])

## Function to preprocess input image

In [14]:
def preprocess_image(img):
    # Resize the image to (256, 256)
    img_resized = cv2.resize(img, (256, 256))
    # Expand the dimensions to make it compatible with model input shape
    img_expanded = np.expand_dims(img_resized / 255.0, axis=0)
    return img_expanded

## Function to predict smile

In [15]:
def predict_smile(img):
    # Preprocess the image
    processed_img = preprocess_image(img)
    # Make prediction using your model
    prediction = model.predict(processed_img)
    # Return the prediction
    return prediction

## Starting the video stream to capture the face

In [7]:
# Open video stream from the webcam
cap = cv2.VideoCapture(0)  # 0 for the default camera, you can change it if you have multiple cameras
# Initialize MTCNN
mtcnn = MTCNN()
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame from BGR to RGB (MTCNN expects RGB)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect faces using MTCNN
    boxes, probs, landmarks = mtcnn.detect(rgb_frame, landmarks=True)

    # Draw bounding boxes and landmarks
    if boxes is not None:
        for box, landmark in zip(boxes, landmarks):
            # Draw bounding box
            cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 255, 0), 2)
            # Draw landmarks
            for point in landmark:
                cv2.circle(frame, (int(point[0]), int(point[1])), 2, (0, 0, 255), -1)

            # Crop the detected face
            face_img = frame[int(box[1]):int(box[3]), int(box[0]):int(box[2])]

            prediction = predict_smile(face_img)

            # Display prediction (e.g., draw text on the frame)
            if prediction > 0.5:
                cv2.putText(frame, 'Smiling', (int(box[0]), int(box[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else:
                cv2.putText(frame, 'Not Smiling', (int(box[0]), int(box[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255),
                            2)

    # Display the resulting frame
    cv2.imshow('Face Detection with MTCNN', frame)

    # Exit loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()